In [ ]:
pip install openai

In [2]:
import pandas as pd
import os
#import openai
from openai import OpenAI

In [3]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

# Import

In [ ]:
df_aut = pd.read_csv('data/authors_publications_full_lang.tsv', sep='\t')
df_aut = df_aut[(df_aut['DB']=='DIALNET') & (df_aut['Lang']=='es')]
df_aut = df_aut[['CODIGO OBRA DIALNET', 'TITULO']].drop_duplicates()
df_aut

In [6]:
df_aut['TITULO_n'] = df_aut['TITULO'].str.split().str.len()
df_aut['TITULO_cum'] = df_aut['TITULO_n'].cumsum()
df_aut['chunks'] = df_aut['TITULO_cum']//700

In [7]:
client = OpenAI(api_key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo")

    return response.choices[0].message.content

In [ ]:
chunk_list = df_aut[~df_aut['chunks'].isin(titles_eng['chunks'].tolist())]['chunks'].drop_duplicates().tolist()
len(chunk_list)

In [ ]:
for i in chunk_list:
    print('\r'+str(i), end='')
    prompt = "I have a table of titles in Spanish of scientific publications from various humanities specialties. The first column is a publication identifier, and the second is the title. I want all the titles translated into English. Please keep in mind that some titles may contain specific references to Spain or Spanish culture, and these should be maintained in the translation. I want you to return a table to me with the publication identifier and the translated title. Separate the columns with tabs (\t) and each row with a newline (\n). This is very important, as it is part of a research project. Here is the table:"+'\n'.join(df_aut[df_aut.chunks==i]['CODIGO OBRA DIALNET']+'\t'+df_aut[df_aut.chunks==i]['TITULO'])
    response = get_completion(prompt)
    titles_eng_aux = pd.DataFrame([x.split(sep='\t') for x in response.split(sep='\n')]).rename({0:'CODIGO OBRA DIALNET', 1:'TITULO'}, axis=1)
    titles_eng_aux['chunks'] = i

    if df_aut[df_aut.chunks==i].shape[0] != titles_eng_aux.shape[0]:
      print(' ERROR')

    titles_eng = pd.concat([titles_eng, titles_eng_aux])
    titles_eng.to_csv('data/titles_eng.tsv', sep='\t', encoding='UTF-8', index=False)